<a href="https://colab.research.google.com/github/bhar-gav/topic-specific-analysis-of-sentences/blob/main/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/drive")

import os
print(os.getcwd())
datasets="/content/drive/My Drive/Colab Notebooks/final-year/sentence-analysis/"
os.chdir(datasets)
print(os.getcwd())

Mounted at /content/drive
/content
/content/drive/My Drive/Colab Notebooks/final-year/sentence-analysis


In [5]:
import pandas as pd

df1=pd.read_csv("datasets/un-general-debates.csv")
df2=pd.read_json("datasets/science-articles.json")
df3=pd.read_json("datasets/environment-articles.json")
df4=pd.read_json("datasets/sports-articles.json")

dfs=[df1,df2,df3,df4]
dataset_names=["UN Speeches","Science Articles","Environment Articles","Sports Articles"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
for i, df in enumerate (dfs,1):
  print(f"\n Dataset : {dataset_names[i-1]}")
  print(df.head())


 Dataset UN Speeches
   session  year country                                               text
0       44  1989     MDV  ﻿It is indeed a pleasure for me and the member...
1       44  1989     FIN  ﻿\nMay I begin by congratulating you. Sir, on ...
2       44  1989     NER  ﻿\nMr. President, it is a particular pleasure ...
3       44  1989     URY  ﻿\nDuring the debate at the fortieth session o...
4       44  1989     ZWE  ﻿I should like at the outset to express my del...

 Dataset Science Articles
                                                link  \
0  https://www.huffpost.com/entry/fuel-leak-ruins...   
1  https://www.huffpost.com/entry/nasa-artemis-mo...   
2  https://www.huffpost.com/entry/china-rocket-de...   
3  https://www.huffpost.com/entry/arthurs-stone-e...   
4  https://www.huffpost.com/entry/james-webb-spac...   

                                            headline category  \
0  Fuel Leak Ruins NASA's 2nd Shot At Launching N...  SCIENCE   
1  'Safety Is Always First':

In [8]:
pip install nltk gensim pyldavis


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models

# Load your DataFrame
df = pd.DataFrame()  # Create an empty DataFrame
df['speech'] = df1['text'].fillna('').astype(str)  # Ensure 'text' column exists

# Set of stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove punctuation and stopwords
    return tokens

# Apply preprocessing to the 'speech' column
df['tokens'] = df['speech'].apply(preprocess)
df = df[df['tokens'].map(len) > 0]  # Keep only rows with tokens

# Create a dictionary and corpus for LDA
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in df['tokens']]

# Fit the LDA model
num_topics = 5  # Adjust based on your dataset
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print the topics and their keywords
print("Topics and Keywords:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

# Get the topics and their keywords for easier visualization
topics = lda_model.show_topics(formatted=False)
topics_keywords = {idx: [word for word, _ in words] for idx, words in topics}

# Print the keywords for each topic
print("\nKeywords for each topic:")
for topic, keywords in topics_keywords.items():
    print(f"Topic {topic}: {', '.join(keywords)}")

# Visualize the topics
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Topics and Keywords:
Topic 0: 0.012*"countries" + 0.012*"international" + 0.011*"united" + 0.009*"nations" + 0.009*"people" + 0.009*"world" + 0.008*"states" + 0.008*"peace" + 0.006*"economic" + 0.005*"africa"
Topic 1: 0.014*"nations" + 0.014*"development" + 0.012*"united" + 0.009*"countries" + 0.009*"international" + 0.007*"world" + 0.006*"global" + 0.006*"must" + 0.005*"economic" + 0.005*"also"
Topic 2: 0.012*"international" + 0.010*"countries" + 0.008*"world" + 0.007*"nations" + 0.007*"peace" + 0.007*"economic" + 0.007*"development" + 0.006*"country" + 0.006*"must" + 0.005*"united"
Topic 3: 0.018*"united" + 0.018*"nations" + 0.015*"international" + 0.011*"security" + 0.008*"peace" + 0.007*"council" + 0.006*"world" + 0.006*"development" + 0.006*"states" + 0.006*"also"
Topic 4: 0.012*"world" + 0.010*"people" + 0.008*"nations" + 0.008*"united" + 0.008*"us" + 0.007*"must" + 0.007*"one" + 0.005*"peace" + 0.005*"would" + 0.005*"war"

Keywords for each topic:
Topic 0: countries, internation